In [1]:
import torch as th
import os
import sys
import pandas as pd
import gc

from src.model import GraphemeClassifier
from src.dataset import DataModule
from src.config import Config

Global seed set to 2021
Global seed set to 2021


## Setup dataset

In [2]:
train_df = pd.read_csv(os.path.join(Config.data_dir, 'train.csv'))

dm = DataModule(
    df=train_df, 
    frac=1, 
    validation_split=.2, 
    train_batch_size=Config.train_batch_size, 
    test_batch_size=Config.test_batch_size
)

dm.setup()

[INFO] Training on 160672 samples
[INFO] Validating on 40168 samples


### Modeling

In [3]:
model = GraphemeClassifier(
    base_encoder= 'resnext50_32x4d',
    arch_from = 'timm',
    vowels_class_weight=None,
    g_root_class_weight=None,
    consonant_class_weight=None,
    dropout=0.2,
    lr=0.01,
    pretrained=False
)

model

GraphemeClassifier(
  (extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track

In [4]:
th.cuda.empty_cache()

for data in dm.train_dataloader():
    images = data['image']
    
    model = model.cuda()
    logits = model(images.unsqueeze(1).cuda())
    
    g_logits = logits["g_logits"]
    v_logits = logits['v_logits']
    c_logits = logits["c_logits"]
    
    print(f"g_logits : {g_logits.shape}")
    print(f"v_logits : {v_logits.shape}")
    print(f"c_logits : {c_logits.shape}")
    
    
    th.cuda.empty_cache()
    del model
    del g_logits
    del v_logits
    del c_logits
    del images
    
    gc.collect()
    
    break


g_logits : torch.Size([32, 168])
v_logits : torch.Size([32, 11])
c_logits : torch.Size([32, 7])
